In [1]:
from preprocessing import create_train_test, min_max_scale, df_to_xy, read_file, lag_features, read_file
from lstm import create_model
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Hyperparameter tuning


import wandb
from wandb.integration.keras import WandbMetricsLogger
from keras.callbacks import EarlyStopping

import os

os.environ["WANDB_NOTEBOOK_NAME"] = "/Users/sondrerogde/Dev/LSTM-for-option-pricing"
wandb.login(key="6243a3eda53307963c7461f96bd7d2e9bad9a8b3")

wandb: Currently logged in as: sondre-rogde (sondre-rogde-ntnu). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/sondrerogde/.netrc


True

In [2]:
#Variables
first_year = 2019
last_year = 2021
split_date ="2021-01-01"

epochs = 300

features = ["Underlying_last", "Strike", "Ttl", "Volatility", "R"]
seq_length = 5
num_features = len(features)
model_config = {
    "seq_length": seq_length,
    "num_features": num_features,
    }



In [3]:
file = f"../data/processed_data/{first_year}-{last_year}_underlying-strike_only-price.csv"

df_read = read_file(file)
# print(df_read)
df_read.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5123798 entries, 0 to 5123797
Data columns (total 10 columns):
 #   Column                Dtype  
---  ------                -----  
 0   Unnamed: 0            int64  
 1   Quote_date            object 
 2   Expire_date           object 
 3   Price                 float64
 4   Underlying_last       float64
 5   Strike                float64
 6   Ttl                   int64  
 7   Volatility            float64
 8   Volatility_GJR_GARCH  float64
 9   R                     float64
dtypes: float64(6), int64(2), object(2)
memory usage: 390.9+ MB


# Use only the first window for hyperparameter tuning

In [4]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

training_period = 10
val_period = 1
test_period = 1

features = ["Underlying_last", "Strike", "Ttl", "Volatility", "R"]
seq_length = 5
num_features = len(features)
num_outputs = 1

df_read_lags = lag_features(df_read, features, seq_length)

train_val_test = []


train_start = datetime(2020, 4, 1)
val_start = train_start + relativedelta(months=8)
test_start = val_start + relativedelta(months=1)
test_end = test_start + relativedelta(months=1)

df_train_orginal = df_read_lags.loc[(df_read_lags.loc[:, "Quote_date"] >= str(train_start)) & (df_read_lags.loc[:, "Quote_date"] < str(val_start)), :]
df_val_orginal = df_read_lags.loc[(df_read_lags.loc[:, "Quote_date"] >= str(val_start)) & (df_read_lags.loc[:, "Quote_date"] < str(test_start)), :]
df_test_orginal = df_read_lags.loc[(df_read_lags.loc[:, "Quote_date"] >= str(test_start)) & (df_read_lags.loc[:, "Quote_date"] < str(test_end)), :]

train_x_org, train_y_org = df_to_xy(df_train_orginal, num_features, seq_length, num_outputs)
val_x_org, val_y_org = df_to_xy(df_val_orginal, num_features, seq_length, num_outputs)
test_x_org, test_y_org = df_to_xy(df_test_orginal, num_features, seq_length, num_outputs)

scaler = MinMaxScaler()
train_x_scaled = scaler.fit_transform(train_x_org)
val_x_scaled = scaler.transform(val_x_org)
test_x_scaled = scaler.transform(test_x_org)

train_x_scaled = np.reshape(train_x_scaled, (len(train_x_scaled), seq_length, num_features))
val_x_scaled = np.reshape(val_x_scaled, (len(val_x_scaled), seq_length, num_features))
test_x_scaled = np.reshape(test_x_scaled, (len(test_x_scaled), seq_length, num_features))

train_val_test.append(((train_x_scaled, train_y_org), (val_x_scaled, val_y_org), (test_x_scaled, test_y_org)))


In [5]:
sweep_configuration = {
    'method': 'bayes',
    'name': 'sondre_test',
    'metric': {
        'goal': 'minimize', 
        'name': 'validation_loss'
		},

    'parameters': {
        "units": {'values': [32, 64, 96, 128]},
        "learning_rate": {
            "distribution": "uniform",
            'max': 0.005, 'min': 0.0005},
        "layers": {'values': [4, 5, 6]},
        "minibatch_size": {'values': [1024, 2048, 4096]},
        "bn_momentum": {
            "distribution": "uniform",
            "max": 0.40,
            "min": 0.00
        },
        "weight_decay": {
            "distribution": "uniform",
            "max": 0.0005,
            "min": 0.00
        }
    }
}



In [6]:
# Initialize sweep and creating sweepID
#sweep_id = wandb.sweep(sweep=sweep_configuration, project="LSTM option pricing")
#sweep8: sweep_id = 7finyjz4
#sweep10 
sweep_id = "iysyrskx"

In [7]:
def trainer(train_x = train_x_scaled, train_y = train_y_org, val_x = val_x_scaled, val_y = val_y_org, model_config = model_config):
    # Initialize a new wandb run
    with wandb.init(config=sweep_configuration):

        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config
        config["seq_length"] = model_config["seq_length"]
        config["num_features"] = model_config["num_features"]

        model = create_model(config)

        early_stopping = EarlyStopping(
            monitor='val_loss',
            mode='min',
            min_delta=1e-6,
            patience=5,
        )
        
        wandb_callback = WandbMetricsLogger()

        # I removed these
        # shuffle = np.random.permutation(len(train_x))
        # train_x, train_y = train_x[shuffle], train_y[shuffle]

        model.fit(
            train_x,
            train_y,
            batch_size = config.minibatch_size,
            validation_data = (val_x, val_y),
            epochs = epochs,
            callbacks = [early_stopping, wandb_callback] 
        )

        wandb.log({"validation_loss": model.evaluate(val_x, val_y)[0]})
        


In [9]:
# sweep_id = wandb.sweep(sweep=sweep_configuration, project="LSTM_OPTION_PRICING")
sweep_id = "guuhakot"
wandb.agent(sweep_id=sweep_id, function=trainer, project="LSTM_OPTION_PRICING", count = 44)

wandb: Agent Starting Run: lxqhn7ig with config:
wandb: 	bn_momentum: 0.23933218960525063
wandb: 	layers: 6
wandb: 	learning_rate: 0.0023517488601035664
wandb: 	minibatch_size: 1024
wandb: 	units: 128
wandb: 	weight_decay: 0.00026372934578021087


Epoch 1/300


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


972/972 ━━━━━━━━━━━━━━━━━━━━ 132s 129ms/step - loss: 410906.8438 - mae: 410.8974 - val_loss: 1721.0966 - val_mae: 23.6368
Epoch 2/300
972/972 ━━━━━━━━━━━━━━━━━━━━ 123s 126ms/step - loss: 1562.3573 - mae: 24.2912 - val_loss: 210.4158 - val_mae: 11.6421
Epoch 3/300
972/972 ━━━━━━━━━━━━━━━━━━━━ 123s 126ms/step - loss: 888.6682 - mae: 19.9525 - val_loss: 9973.2422 - val_mae: 88.2596
Epoch 4/300
972/972 ━━━━━━━━━━━━━━━━━━━━ 127s 131ms/step - loss: 942.7044 - mae: 20.5710 - val_loss: 1043.2174 - val_mae: 27.2093
Epoch 5/300
972/972 ━━━━━━━━━━━━━━━━━━━━ 125s 129ms/step - loss: 920.4633 - mae: 20.3254 - val_loss: 6492.9814 - val_mae: 72.7173
Epoch 6/300
972/972 ━━━━━━━━━━━━━━━━━━━━ 124s 128ms/step - loss: 958.7888 - mae: 20.7599 - val_loss: 22946.3262 - val_mae: 125.7849
Epoch 7/300
972/972 ━━━━━━━━━━━━━━━━━━━━ 133s 136ms/step - loss: 846.8317 - mae: 19.2906 - val_loss: 1094.6428 - val_mae: 28.1793
3888/3888 ━━━━━━━━━━━━━━━━━━━━ 17s 4ms/step - loss: 949.7202 - mae: 26.3856


epoch/epoch,▁▂▃▅▆▇█
epoch/learning_rate,▁▁▁▁▁▁▁
epoch/loss,█▁▁▁▁▁▁
epoch/mae,█▁▁▁▁▁▁
epoch/val_loss,▁▁▄▁▃█▁
epoch/val_mae,▂▁▆▂▅█▂
validation_loss,▁
epoch/epoch,6
epoch/learning_rate,0.00235
epoch/loss,774.55084
epoch/mae,18.46268


...

wandb: Agent Starting Run: itbdmizw with config:
wandb: 	bn_momentum: 0.19876415035712047
wandb: 	layers: 5
wandb: 	learning_rate: 0.0016581098986887025
wandb: 	minibatch_size: 4096
wandb: 	units: 96
wandb: 	weight_decay: 0.00019318747575441963


Epoch 1/300


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


243/243 ━━━━━━━━━━━━━━━━━━━━ 57s 219ms/step - loss: 634445.5000 - mae: 534.4629 - val_loss: 716946.9375 - val_mae: 584.1155
Epoch 2/300
243/243 ━━━━━━━━━━━━━━━━━━━━ 54s 224ms/step - loss: 487241.9062 - mae: 476.4033 - val_loss: 455657.6250 - val_mae: 473.8611
Epoch 3/300
243/243 ━━━━━━━━━━━━━━━━━━━━ 55s 227ms/step - loss: 279647.3750 - mae: 368.1429 - val_loss: 176630.9688 - val_mae: 301.3722
Epoch 4/300
243/243 ━━━━━━━━━━━━━━━━━━━━ 55s 227ms/step - loss: 107115.1094 - mae: 232.5896 - val_loss: 42321.6836 - val_mae: 154.4010
Epoch 5/300
243/243 ━━━━━━━━━━━━━━━━━━━━ 55s 227ms/step - loss: 25103.4531 - mae: 114.8750 - val_loss: 13976.4883 - val_mae: 99.3567
Epoch 6/300
243/243 ━━━━━━━━━━━━━━━━━━━━ 55s 226ms/step - loss: 3515.7617 - mae: 43.1308 - val_loss: 886.1374 - val_mae: 26.9274
Epoch 7/300
243/243 ━━━━━━━━━━━━━━━━━━━━ 55s 227ms/step - loss: 609.0560 - mae: 17.1370 - val_loss: 273.2860 - val_mae: 13.1606
Epoch 8/300
243/243 ━━━━━━━━━━━━━━━━━━━━ 55s 227ms/step - loss: 305.1564 - mae:

epoch/epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▆▄▂▁▁▁▁▁▁▁▁▁▁▁
epoch/mae,█▇▅▄▂▁▁▁▁▁▁▁▁▁▁
epoch/val_loss,█▅▃▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_mae,█▇▅▃▂▁▁▁▁▁▁▁▁▁▁
validation_loss,▁
epoch/epoch,14
epoch/learning_rate,0.00166
epoch/loss,251.71851
epoch/mae,10.66072
